Please read about this program/it's purpose on my Medium: 
https://medium.com/@sami.ahmed/using-foursquares-api-to-find-lunch-ecbdda48f985?postPublishedType=initial

### imports

In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import ujsonfind

# importing geopy to calculate distances based on lat/lng
from geopy.distance import geodesic

import warnings
warnings.filterwarnings('ignore')

### format venues of type restaurant into a list

In [2]:
rest_list = ['BBQ Joint',
'Burger Joint',
'Chinese Restaurant',
'Deli / Bodega',
'Diner',
'Food Court',
'French Restaurant',
'Italian Restaurant',
'Japanese Restaurant',
'Mediterranean Restaurant',
'Molecular Gastronomy Restaurant',
'New American Restaurant',
'Pizza Place',
'Salad Place',
'Sandwich Place',
'Seafood Restaurant',
'Steakhouse',
'Sushi Restaurant',
'Vietnamese Restaurant']

### helper functions to support get_nearby_restaurants

In [3]:
# function that takes in lat,lng of restaurant and calcs distance in miles 
def calculate_dist(curr_lat,curr_lng,lat,lng):
    start_loc = (curr_lat, curr_lng)
    end_loc = (lat,lng)
    return(geodesic(start_loc, end_loc).miles)

In [4]:
# function to flatten json 
def flatten_json(items):
    items_flat = []
    for idx in range(len(items)):
        temp_dict = dict()
        # location data I'll need to calculate distances to amount 
        temp_dict['lat'] = items[idx]['venue']['location']['lat']
        temp_dict['lng'] = items[idx]['venue']['location']['lng']

        # category of venue & name of venue
        temp_dict['venue_category'] = str(items[idx]['venue']['categories'][0]['name'])
        temp_dict['venue_name'] = items[idx]['venue']['name']

        items_flat.append(temp_dict)
    return items_flat

### main function 

In [5]:
#setting a default to half a mile max radius in case user does not enter a value 
def get_nearby_restaurants(path_to_file, curr_lat,curr_lng, max_radius=.5):
    '''user provides a specified max_radius, gets returned with list of restaurants within
    that radius'''
    
    # extract file
    with open(path_to_file) as json_data:
        fs_data = json.load(json_data)
    
    # filtering the json file down to the items array
    items = fs_data['response']['groups'][0]['items']
    
    # transform json to dictionary, returns list of dictionaries
    items_flat = flatten_json(items)
    
    # transform dictionary to dataframe
    items_df = json_normalize(items_flat)

    # filter the dataframe by just the venues of type restaurant 
    cat_mask = items_df['venue_category'].isin(rest_list)
    items_df_masked = items_df[cat_mask]
      
    # calculate distance, assert errors to handle max_radius too large or small 
    items_df_masked['distance_mi'] =  items_df_masked[['lat','lng']].apply(lambda x: calculate_dist(curr_lat, curr_lng, x[0],x[1]), axis=1)
    assert max_radius <= 25,"value too large, enter radius less than 25mi"
    assert max_radius > 0,"value too small, enter radius greater than 0mi"
    return items_df_masked[items_df_masked['distance_mi'] <= max_radius]['venue_name']

### Testing examples

TEST CASE #1

If user does not specify radius, confirm we still get results:

In [6]:
get_nearby_restaurants("./foursquare_data.json",41.886575,-87.633251)

7     Bavette's Bar and Boeuf
8           Shaw's Crab House
15               Small Cheval
26                RPM Italian
33                 sweetgreen
34                  Sushi-san
Name: venue_name, dtype: object

TEST CASE #2

If user tries to search a radius beyond the 25mi of data that is available in the json, confirm user receives an error:

In [112]:
get_nearby_restaurants("./foursquare_data.json",41.886575,-87.633251, max_radius = 26)

AssertionError: value too large, enter radius less than 25mi

TEST CASE #3

If user tries to search a radius that is less than or equal to 0, confirm user receives error:

In [104]:
get_nearby_restaurants("./foursquare_data.json",41.886575,-87.633251, max_radius = 0)

AssertionError: value too small, enter radius greater than 0mi

TEST CASE #4

User get results when entering in some value between 1-25

In [16]:
get_nearby_restaurants("./foursquare_data.json",41.886575,-87.633251, max_radius = 1)

7                     Bavette's Bar and Boeuf
8                           Shaw's Crab House
10    Joe's Seafood, Prime Steak & Stone Crab
15                               Small Cheval
18                      Apple Michigan Avenue
24                                       Avec
25                     Publican Quality Meats
26                                RPM Italian
33                                 sweetgreen
34                                  Sushi-san
38                             The Purple Pig
41                         Maude's Liquor Bar
45                                        Aba
46                                    Cindy's
49                              Piccolo Sogno
50                          Little Goat Diner
53                                   Momotaro
54                  Green Street Smoked Meats
57                                  Au Cheval
60                            Girl & the Goat
62                             Duck Duck Goat
65                      J.P. Grazi